# FLEXible tutorial: Text classification using Tensorflow

FLEXible is a library to federate models. We offer the tools to load and federate data or to load federated data, and the tools to create a federated environment. The user must define the model and the *communication primitives* to train the model in a federated environment. This primitives can be expressed in the following steps:
- initialization: Initialize the model in the server.
- deplot model: Deploy the model to the clients.
- training: Define the train function.
- collect the weights: Collect the weights of the clients params to aggregate them later.
- aggregate the weights: Use an aggregation method to aggregte the collected weights.
- deploy model: Deploy the model with the updated weights to the clients.
- evaluate: Define the evaluate function.

In this notebook, we show how to implement this primitives and how to use FLEXible in orther to federate a model using TensorFlow. In this way, we will train a model using multiple clients, but without sharing any data between clients. We will follow this [tutorial](https://www.tensorflow.org/hub/tutorials/tf2_text_classification#build_the_model) from the TensorFlow tutorials for text classification. 

## Setup

In [ ]:
from copy import deepcopy
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

As usual in every experiment, the first step is to load the dataset we will use. In this case we will use the dataset **imdb_reviews** for a supervised text classification model

## Download the IMBD dataset

As used in the tutorial from TensorFlow, we will use the IMBD dataset. This dataset contains reviews about movies, and the *sentiment* associated to them.

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"])

# 1) From centralized data to federated data
Firstly and foremost, we need to encapsulare our centralized dataset as numpy arrays in a Dataset, to split it for every federated client.
As we are using a centrilized dataset, we have to federate it. To federate the data we need to create a basic data object for FLEXible that is called **Dataset**. To create a  **Dataset** we use the method **from_tfds_dataset** which extracts the features and labels of the dataset and transform them into numpy arrays.

In [ ]:
from flex.data import Dataset

flex_data = Dataset.from_tfds_text_dataset(train_data, X_columns='text', label_columns='label')

In order to federate our dataset, we need to specify how we want to split it among clients in a ``FedDatasetConfig`` object. For this case we want to split it evenly between 2 clients, that is, an iid distribution. To apply our config to our centralized dataset, we use ``FedDataDistribution.from_config``. A more complete description of the configuration options of ``FedDatasetConfig`` to federate a dataset can be found in the documentation.

In [ ]:
from flex.data import FedDatasetConfig, FedDataDistribution

config = FedDatasetConfig(seed=0)
config.n_clients = 2
config.replacement = False # ensure that clients do not share any data
config.client_names = ['client1', 'client2'] # Optional
flex_dataset = FedDataDistribution.from_config(centralized_data=flex_data, config=config)

However, there is a shortcut, if we want to split the dataset iid between the clients we can directly use ``FedDataDistribution.iid_distribution`` with the number of clients and our centralized data stored in a ``Dataset``. Note that in this case the name of the clients are generated automatically: client number ``i`` gets id: ``f"client_{i}"``.

In [ ]:
from flex.data import FedDataDistribution

flex_dataset = FedDataDistribution.iid_distribution(flex_data, n_clients=2)

# 2) Federating a model with FLEXible

Once we've federated the dataset, we have to create the FlexPool. The FlexPool class simulates a real-time scenario for federated learning, so it is in charge of the communications across the actors. The class FlexPool will assign to each actor a role (client, aggregator, server), so they can communicate during the training phase.

Please, check the notebook about the actors (TODO: Hacer notebook actores y sus relaciones) to know more about the actors and their relationships in FLEXible.

To create a Pool of actors, we need to have a federated dataset, like we've just done, and the model to initialize in the server side, because the server will send the model to the clients so they can train the model. As we have the federated dataset (flex_dataset), we will now create the model.

In this case, we will use a model from the tensorflow hub, so we dont have to worry about coding it. We also consider a federated setup commonly know as client server architecture, where a server orchestrates the training of federated clients in every round.

In the following, we create a client server architecture and provide a function to initialize the server model.

In [ ]:
def define_model():
    # model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
    model = "https://tfhub.dev/google/nnlm-en-dim50/2"
    hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',
                    loss=tf.losses.BinaryCrossentropy(from_logits=True),
                    metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
    return model

In [ ]:
def initialize_server_model(server_flex_model, *args):
    print("Initializing server model.")
    model = define_model()
    server_flex_model["optimizer"] = deepcopy(model.optimizer)
    server_flex_model["loss"] = deepcopy(model.loss)
    server_flex_model["metrics"] = deepcopy(model.compiled_metrics._metrics)
    server_flex_model["model"] = model

In [ ]:
from flex.pool import FlexPool

flex_pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model)
clients = flex_pool.clients
server = flex_pool.servers
print(f"Server node is indentified by {server.actor_ids}")
print(f"Client nodes are identified by {clients.actor_ids}")

We have to create the function that will deploy the model to the clients. 

In [ ]:
def deploy_model_to_clients(server_flex_model, clients_model, *args, **kwargs):
    from flex.model import FlexModel
    for client in clients_model:
        weights = server_flex_model["model"].get_weights()
        model = tf.keras.models.clone_model(server_flex_model["model"])
        model.set_weights(weights)
        model.compile(
            optimizer=server_flex_model["optimizer"],
            loss=server_flex_model["loss"],
            metrics=server_flex_model["metrics"],
        )
        client_flex_model = FlexModel()
        client_flex_model['model'] = model
        clients_model[client].update(client_flex_model)

To work in an easier way, FlexPool let the use to have organized pools, such as clients, aggregators or servers. This helps to understand how we are connecting the actors.

In [ ]:
clients = flex_pool.clients
server = flex_pool.servers

To apply all the primitives, such as the deploy step, we will use the **map** function from *FlexPool*. The map function works in the following way: the pool that calls the function map, is the one that will send a message to the destiny pool. If we don't specify it to any pool, no destiny pool, it will "send" the message to the same pool that it's calling the map function. This is needed if we want to tell the clients to train/evaluate the model.

In [ ]:
server.map(deploy_model_to_clients, clients)

Once the model is deployed on the clients, is time to create the training function.

In [ ]:
def train(client_model, client_data, batch_size=256, epochs=1):
    print("Training model at client.")
    print(client_model)
    model = client_model['model']
    X_data = client_data.X_data
    y_data = client_data.y_data
    history = model.fit(X_data, y_data, epochs=epochs, batch_size=batch_size, verbose=0)

Now we will train the model in the clients side. We will use the *map function* to tell the clients to train the model, and, to do so, we just need to use this function from the clients pool.

In [ ]:
clients.map(train, batch_size=512, epochs=1)

Now that we have trained the model we have to aggregate the weights. To do so, clients will send the weights to the aggregator, and she will perform the aggregation step. For the tutorial, we will implement the FevAvg aggregation mechanism. That is, the aggreation step is split in two steps, 1) for collecting the weights from each client and 2) for averaging them.

First, we select the aggregator, which in this case is the same as the server, because in the client server architecture, the server is also an aggregator.

In [ ]:
aggregator = flex_pool.aggregators
aggregator.actor_ids

In [ ]:
def collect_weights(aggregator_model, clients_model, **kwargs):
    print("Collecting weights.")
    if 'weights' not in aggregator_model:
        aggregator_model['weights'] = []
    for k in clients_model:
        client_weights = clients_model[k]['model'].get_weights()
        aggregator_model['weights'].append(client_weights)

In [ ]:
# clients.map(collect_weights, aggregator)
aggregator.map(collect_weights, clients)

In [ ]:
def aggregate_weights(agg_model, *args):
    print("Aggregating weights")
    averaged_weights = np.mean(np.array(agg_model['weights']), axis=0)
    agg_model["model"].set_weights(averaged_weights)
    agg_model["weights"] = []

In [ ]:
aggregator.map(aggregate_weights)

Now it's turn from the server to update the weights from the clients models and then evaluate the model.

In [ ]:
def deploy_global_model_to_clients(server_model, clients_models, *args, **kwargs):
    print("Deploying the global model on the clients.")
    aggregated_weights = server_model['model'].get_weights()
    for client_model in clients_models:
        clients_models[client_model]['model'].set_weights(aggregated_weights)

In [ ]:
server.map(deploy_global_model_to_clients, clients)

And now, we can evaluate the model with the test set that we prepared at the begining of the notebook.

In [ ]:
def evaluate_model(model, data, *args, **kwargs):
    model = model['model']
    if data is not None:
        print("Evaluating model at client.")
        results_local = model.evaluate(data.X_data, data.y_data, verbose=0)
        print(f"Results at client on client's data: {results_local}")
    else:
        print("Evaluating model at server")
    results = model.evaluate(kwargs['test_examples'], kwargs['test_labels'], verbose=0)
    print(f"Results on test data: {results}")

In [ ]:
# test_examples, test_labels = test_data
test_examples = Dataset.from_tfds_text_dataset(test_data, X_columns='text', label_columns='label')

In [ ]:
server.map(evaluate_model, test_examples=test_examples.X_data, test_labels=test_examples.y_data)

In [ ]:
clients.map(evaluate_model, test_examples=test_examples.X_data, test_labels=test_examples.y_data)

# Putting it all together

You just have trained a model for 1 round using FLEXible. Now, you could set up all together in a function and iterate for multiple rounds.

In [ ]:
def train_n_rounds(n_rounds, batch_size, epochs):
    pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model)
    pool.servers.map(deploy_model_to_clients, pool.clients)
    for i in range(n_rounds):
        print(f"\nRunning round: {i}\n")
        pool.clients.map(train, batch_size=batch_size, epochs=epochs)
        pool.clients.map(evaluate_model, test_examples=test_examples.X_data, test_labels=test_examples.y_data)
        pool.aggregators.map(collect_weights, pool.clients)
        pool.aggregators.map(aggregate_weights)
        pool.servers.map(deploy_global_model_to_clients, pool.clients)
        pool.servers.map(evaluate_model, test_examples=test_examples.X_data, test_labels=test_examples.y_data)

In [ ]:
train_n_rounds(n_rounds=4, batch_size=512, epochs=2)

### END
Congratulations, now you know how to train a model using FLEXible for multiples rounds. Remember that it's important to first deploy/initialize the model on the clients, so you can run the rounds without problem!